# 🪙 G-Research Crypto - Starter LGBM Pipeline
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)

---

# [[S] 🪙💲 Proposal for a meaningful LB](https://www.kaggle.com/julian3833/s-proposal-for-submission-common-ground)
# 👆👆👆 A proposal for an agreement to get comparable, non-leaky models

<h3 style="text-align:center; background-color:#FFEECC;padding:40px;border-radius: 30px;">
    <b>Watch out!</b>: the test data is contained in the original train data 😒, so the LB score of <i>0.313</i> is overfitting. The best LB without the overlap, as of today, is <i>0.016</i>. This can be further improved.
    <br><br>
    In this version, the variable `REMOVE_LB_TEST_OVERLAPPING_DATA` controls whether to keep only data from before the placeholder <i>test.csv</i> for training or use the full <i>train.csv</i>. 
    <br><br>
    I wrote a simple notebook (~18 lines) fully exploiting this fact <a href="https://www.kaggle.com/julian3833/g-research-using-the-overlap-fully-lb-0-99">here</a>.
    
</h3>

A copy of this notebook without exposition to the "leaky" data: [[S]🪙💲G-Research - Starter LGBM [LB=0.017]](https://www.kaggle.com/julian3833/s-g-research-starter-lgbm-lb-0-017/)



# For more information:
* __[Watch out!: test LB period is contained in the train csv](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/285505) (topic)__
* __[🪙💲 G-Research- Using the overlap fully [LB=0.99]](https://www.kaggle.com/julian3833/g-research-using-the-overlap-fully-lb-0-99) (notebook)__
* __[Meaningful submission scores / sharing the lower boundary of public test data](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/285289) (topic)__



---


### Just a simple pipeline going from zero to a valid submission

We train one `LGBMRegressor` for each asset over a very very naive set of features (the input dataframe `['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']`), we get the predictions correctly using the iterator and we submit. No validation for now, no cross validation... nothing at all lol: just the bare pipeline!


## Please _DO_ upvote if you find this useful!


__Changelog__

__V20__: __V15__ with `10` estimators.

[__V16__](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline?scriptVersionId=78762326): Fooling around with the leakage again. `REMOVE_LB_TEST_OVERLAPPING_DATA=False` and set `n_estimators=3000`. `LB=0.313`


[__V15__](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline?scriptVersionId=78757083) : Added flag `REMOVE_LB_TEST_OVERLAPPING_DATA` to keep only train data that is _before_ the placeholder test data (`'2021-06-13 00:00:00'`). `LB=0.016`. Reduced estimators to `5`

[__V11__](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline?scriptVersionId=78722210): `LB=201`.

[__V5__](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline?scriptVersionId=78663867): Added two non-timely features from the tutorial: `Upper_Shadow` and `Lower_Shadow`



## References:
* [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)
* [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template)
* [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)



# Import and load dfs

References: [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)

In [1]:
import os
import random
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

SEED = 2021

REMOVE_LB_TEST_OVERLAPPING_DATA = True

In [2]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

fix_all_seeds(SEED)

In [3]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


## Keep only values _before_ the LB test set

In [4]:
# Remove the future
if REMOVE_LB_TEST_OVERLAPPING_DATA:
    df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')
    df_train = df_train[df_train['datetime'] < '2021-06-13 00:00:00']

In [5]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


# Training

## Utility functions to train a model for one asset

In [6]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    return df_feat

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    # TODO: Try different models here!
    model = LGBMRegressor(n_estimators=10)
    model.fit(X, y)
    return X, y, model

## Loop over all assets

In [7]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

Training model for Binance Coin     (ID=0 )
Training model for Bitcoin          (ID=1 )
Training model for Bitcoin Cash     (ID=2 )
Training model for Cardano          (ID=3 )
Training model for Dogecoin         (ID=4 )
Training model for EOS.IO           (ID=5 )
Training model for Ethereum         (ID=6 )
Training model for Ethereum Classic (ID=7 )
Training model for IOTA             (ID=8 )
Training model for Litecoin         (ID=9 )
Training model for Maker            (ID=10)
Training model for Monero           (ID=11)
Training model for Stellar          (ID=12)
Training model for TRON             (ID=13)


In [8]:
# Check the model interface
x = get_features(df_train.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

9.477576309595709e-06

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [9]:
all_df_test = []

env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)
    all_df_test.append(df_test)

    # Send submissions
    env.predict(df_pred)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


Count             1201.000000
Open                 1.478556
High                 1.486030
Low                  1.478000
Close                1.483681
Volume          654799.561103
VWAP                 1.481439
Upper_Shadow         0.002349
Lower_Shadow         0.000556
Name: 0, dtype: float64

,row_id,Target
0,0,-0.000031
1,1,-0.000002
2,2,0.000020
3,3,0.000013
4,4,0.000032
5,5,-0.000004
6,6,0.000072
7,7,-0.000007
8,8,-0.000007
9,9,-0.000017


# df_train and df_test overlap

In [10]:
df_test = pd.concat(all_df_test)
df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')

In [11]:
df_train['datetime'].max()

Timestamp('2021-06-12 23:59:00')

In [12]:
df_test['datetime'].min()

Timestamp('2021-06-13 00:00:00')